# Introduction

In YuC_SITK_1, I presented the installation and running over the most wrapped version of sitk on both simulated data and the tissue data from Nitzan. However, there're some remaining challenges for finishing the pipeline


   - There's no measurement of similarity/cost in the end to evaluate the efficiency of registration
   - The wrapper functions does not accomodate further customization of pyramid level, cost function, etc
   - It was generally hard to tell how well the registration was qualitatively from the overlapping image

In this notebook, I adopted functions from the InsightSoftwareConsortium Tutorial to run through a more parametrized and interative inspection of cost metrics on the Tissue Image

In [4]:
import SimpleITK as sitk

# Utility method that either downloads data from the Girder repository or
# if already downloaded returns the file name for reading from disk (cached data).
%run update_path_to_download_script
from downloaddata import fetch_data as fdata

# Always write output to a separate directory, we don't want to pollute the source directory. 
import os
OUTPUT_DIR = 'Output'

from matplotlib import pyplot as plt
from skimage.io import imread, imsave, imshow
from skimage.transform import resize as imresize
from os.path import join
import imagecodecs

## Utility functions
A number of utility callback functions for image display and for plotting the similarity metric during registration.

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt

from ipywidgets import interact, fixed
from IPython.display import clear_output

# Callback invoked by the interact IPython method for scrolling through the image stacks of
# the two images (moving and fixed).
def display_images(fixed_image_z, moving_image_z, fixed_npa, moving_npa):
    # Create a figure with two subplots and the specified size.
    plt.subplots(1,2,figsize=(10,8))
    
    # Draw the fixed image in the first subplot.
    plt.subplot(1,2,1)
    plt.imshow(fixed_npa[fixed_image_z,:,:],cmap=plt.cm.Greys_r);
    plt.title('fixed image')
    plt.axis('off')
    
    # Draw the moving image in the second subplot.
    plt.subplot(1,2,2)
    plt.imshow(moving_npa[moving_image_z,:,:],cmap=plt.cm.Greys_r);
    plt.title('moving image')
    plt.axis('off')
    
    plt.show()

# Callback invoked by the IPython interact method for scrolling and modifying the alpha blending
# of an image stack of two images that occupy the same physical space. 
def display_images_with_alpha(image_z, alpha, fixed, moving):
    img = (1.0 - alpha)*fixed[:,:,image_z] + alpha*moving[:,:,image_z] 
    plt.imshow(sitk.GetArrayViewFromImage(img),cmap=plt.cm.Greys_r);
    plt.axis('off')
    plt.show()

def display_images_with_alpha_2d(alpha, fixed, moving):
    img = (1.0 - alpha)*fixed[:,:] + alpha*moving[:,:] 
    plt.imshow(sitk.GetArrayViewFromImage(img),cmap=plt.cm.Greys_r);
    plt.axis('off')
    plt.show()
    
# Callback invoked when the StartEvent happens, sets up our new data.
def start_plot():
    global metric_values, multires_iterations
    
    metric_values = []
    multires_iterations = []

# Callback invoked when the EndEvent happens, do cleanup of data and figure.
def end_plot():
    global metric_values, multires_iterations
    
    del metric_values
    del multires_iterations
    # Close figure, we don't want to get a duplicate of the plot latter on.
    plt.close()

# Callback invoked when the IterationEvent happens, update our data and display new figure.
def plot_values(registration_method):
    global metric_values, multires_iterations
    
    metric_values.append(registration_method.GetMetricValue())                                       
    # Clear the output area (wait=True, to reduce flickering), and plot current data
    clear_output(wait=True)
    # Plot the similarity metric values
    plt.plot(metric_values, 'r')
    plt.plot(multires_iterations, [metric_values[index] for index in multires_iterations], 'b*')
    plt.xlabel('Iteration Number',fontsize=12)
    plt.ylabel('Metric Value',fontsize=12)
    plt.show()
    
# Callback invoked when the sitkMultiResolutionIterationEvent happens, update the index into the 
# metric_values list. 
def update_multires_iterations():
    global metric_values, multires_iterations
    multires_iterations.append(len(metric_values))

## Read images

We first read the images, casting the pixel type to that required for registration (Float32 or Float64) and look at them.

In [5]:
data_dir = r'../.././data/van_tissue/'

fix_name = r'img1.tif'
moving_name = r'img2.tif'
result_name = r'result.tif'

fix_path = join(data_dir, fix_name)
moving_path = join(data_dir, moving_name)
result_path = join(data_dir, result_name)

fixedImage = imread(fix_path)
movingImage = imread(moving_path)

In [13]:
fixed_image = sitk.ReadImage(fix_path,sitk.sitkFloat32)
moving_image = sitk.ReadImage(moving_path,sitk.sitkFloat32)
interact(display_images_with_alpha_2d, alpha = (0.0,1.0,0.05),fixed = fixed(fixed_image), moving=fixed(moving_image));

interactive(children=(FloatSlider(value=0.5, description='alpha', max=1.0, step=0.05), Output()), _dom_classes…

## Attempt 1: Repetition of Wrapper Function 

In [14]:
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(fixed_image)
elastixImageFilter.SetMovingImage(moving_image)

parameterMapVector = sitk.VectorOfParameterMap()
parameterMapVector.append(sitk.GetDefaultParameterMap("affine"))
parameterMapVector.append(sitk.GetDefaultParameterMap("bspline"))
elastixImageFilter.SetParameterMap(parameterMapVector)

elastixImageFilter.Execute()

transformed_moving = elastixImageFilter.GetResultImage()
# transformParameterMap = elastixImageFilter.GetTransformParameterMap()

In [15]:
interact(display_images_with_alpha_2d, alpha = (0.0,1.0,0.05),fixed = fixed(fixed_image), moving=fixed(transformed_moving));

interactive(children=(FloatSlider(value=0.5, description='alpha', max=1.0, step=0.05), Output()), _dom_classes…